In [1]:
import pandas as pd, numpy as np
np.set_printoptions(suppress=True)# 关掉科学计数法
import glob
import os
import csv
# 一次性merge多个pct_chg
from functools import reduce
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels import regression

# import tushare as ts
import time, urllib
# ts.set_token('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')
# pro = ts.pro_api('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')

# import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)


# from plotly.graph_objs import Scatter,Layout
# import plotly
# import plotly.offline as py
# import numpy as np
# import plotly.graph_objs as go

# #setting offilne
# plotly.offline.init_notebook_mode(connected=True)

In [31]:
stocks_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
etfs_df = pd.read_csv("filtered_50_etfs_by_vol_20200224.csv")
tickers = list(etfs_df['etf'].str.split(".",expand = True)[0])
#     date_col = 'trade_date'
#     code_col = 'ts_code'
#     price_col = 'close'
date_col = 'date'
code_col = 'code'
price_col = 'close'
ticker_type = 'string'
asset_name = "code"

In [24]:
from scipy.optimize import minimize

class RiskParity(object):

def __init__(self, stocks_path, tickers, 
             date_col, code_col, price_col, 
             ticker_type, asset_name, draw_pie_graph):

    self.path = stocks_path
    self.tickers = tickers
    self.date_col = date_col
    self.code_col = code_col
    self.price_col = price_col
    self.ticker_type = ticker_type
    self.asset_name = asset_name
    self.draw_pie_graph = draw_pie_graph
    self.ticker_df_list = self.get_date_price_code_return_list()
    self.tgt_returns = self.ticker_df_list
    self.tgt_merged_returns = self.merge_dfs_by_ticker(self.tgt_returns, 
                                                       self.date_col)
    self.wts, self.risk = self.get_smart_weight(self.tgt_merged_returns, 
                                                method='risk parity', 
                                                cov_adjusted=False, 
                                                wts_adjusted=False)
    self.df_wts, self.risk_parity_tickers, self.weights = self.get_df_wts()


# Get date_col, price_col, code_col, pct_chg_col
def get_date_price_code_return_list(self):
    # for etf data cols are 'date', 'close', 'code'
    ticker_df_list = []
    for ticker in self.tickers:
        try:
            ticker_df = pd.read_csv(self.path+ticker+".csv")
            ticker_df = ticker_df.sort_values(self.date_col)
            ticker_df = ticker_df[[self.date_col, 
                                   self.price_col, 
                                   self.code_col]]
            ticker_df['pct_chg'] = ticker_df[self.price_col].pct_change()
            ticker_df = ticker_df[[self.date_col, 'pct_chg']].dropna()
            ticker_df.columns = [self.date_col, ticker]
            ticker_df_list.append(ticker_df)
        except Exception as e:
            print(e)
    return ticker_df_list


def merge_dfs_by_ticker(self, ticker_df_list, date_col):
    merged_all = reduce(lambda left, right: pd.merge(left, right, on=date_col), ticker_df_list)
#         merged_all = reduce(merge_df_for_reduce, ticker_df_list)
    merged_all.set_index(self.date_col, inplace=True)
    merged_all.dropna(how="all", axis = 1, inplace = True)
    merged_all.fillna(method="ffill", inplace = True)
    return merged_all


def get_smart_weight(self, pct, method, cov_adjusted, wts_adjusted):
    if cov_adjusted == False:
        #协方差矩阵
        cov_mat = pct.cov()
    else:
        #调整后的半衰协方差矩阵
        cov_mat = pct.iloc[:len(pct)/4].cov()*(1/10.) + pct.iloc[len(pct)/4+1:len(pct)/2].cov()*(2/10.) +\
            pct.iloc[len(pct)/2+1:len(pct)/4*3].cov()*(3/10.) + pct.iloc[len(pct)/4*3+1:].cov()*(4/10.)
    if not isinstance(cov_mat, pd.DataFrame):
        raise ValueError('cov_mat should be pandas DataFrame！')

    omega = np.matrix(cov_mat.values)  # 协方差矩阵

    a, b = np.linalg.eig(np.array(cov_mat)) #a为特征值,b为特征向量
    a = np.matrix(a)
    b = np.matrix(b)
    # 定义目标函数

    def fun1(x):
        tmp = (omega * np.matrix(x).T).A1
        risk = x * tmp/ np.sqrt(np.matrix(x) * omega * np.matrix(x).T).A1[0]
        delta_risk = [sum((i - risk)**2) for i in risk]
        return sum(delta_risk)

    def fun2(x):
        tmp = (b**(-1) * omega * np.matrix(x).T).A1
        risk = (b**(-1)*np.matrix(x).T).A1 * tmp/ np.sqrt(np.matrix(x) * omega * np.matrix(x).T).A1[0]
        delta_risk = [sum((i - risk)**2) for i in risk]
        return sum(delta_risk)

    # 初始值 + 约束条件 
    x0 = np.ones(omega.shape[0]) / omega.shape[0]  
    bnds = tuple((0,None) for x in x0)
    cons = ({'type':'eq', 'fun': lambda x: sum(x) - 1})
    options={'disp':False, 'maxiter':1000, 'ftol':1e-20}


    #------------------问题出在这里------------------
    if method == 'risk parity':
        res = minimize(fun1, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)        
    elif method == 'pc risk parity':
        res = minimize(fun2, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
    #------------------------------------

    else:
        raise ValueError('method error！！！')

    # 权重调整
    if res['success'] == False:
        # print res['message']
        pass
    wts = pd.Series(index=cov_mat.index, data=res['x'])

    if wts_adjusted == True:
        wts[wts < 0.0001]=0.0
        wts = wts / wts.sum()
    elif wts_adjusted == False:
        wts = wts / wts.sum()
    else:
        raise ValueError('wts_adjusted should be True/False！')

    risk = pd.Series(wts * (omega * np.matrix(wts).T).A1 / np.sqrt(np.matrix(wts) * omega * np.matrix(wts).T).A1[0],index = cov_mat.index)
    risk[risk<0.0] = 0.0
    return wts,risk


def get_df_wts(self):
    df_wts = pd.DataFrame(self.wts)
    df_wts.reset_index(inplace = True)
    df_wts.columns = [self.asset_name, 'weight']
    risk_parity_tickers = list(df_wts[self.asset_name])
    weights = list(df_wts['weight'])
    return df_wts, risk_parity_tickers, weights


In [27]:
risk_parity = RiskParity(stocks_path,
                          ['512660','518880'],
                          date_col,
                          code_col,
                          price_col,
                          ticker_type,
                          asset_name,
                          True)

In [29]:
df_wts, risk_parity_tickers, weights = risk_parity.get_df_wts()

In [30]:
df_wts

,code,weight
0,512660,0.309321
1,518880,0.690679
